## Desafio Capim

#### Test Case - Data Analyst

In [ ]:
import pandas as pd

clinics = pd.read_csv('clinics.csv')
subscriptions = pd.read_csv('subscriptions.csv')
activity = pd.read_csv('activity.csv')

#### Análise de Dados Exploratória (EDA)

#### Clinics

In [ ]:
clinics.head()

#### Limpeza e Preparação dos Dados

In [ ]:
date_columns = [
    'CLINIC_CREATED_AT',
    'TRIAL_START_DATE',
    'TRIAL_END_DATE'
]

for col in date_columns:
    clinics[col] = pd.to_datetime(clinics[col])

clinics.info()

In [ ]:
clinics = clinics[~((clinics['BUSINESS_SEGMENTATION'] == 'Rede Homologada') & (clinics['IS_CHAIN_CLINIC'] == True))]

clinics.info()

In [ ]:
dias_da_semana = {
    1: 'Segunda-feira',
    2: 'Terça-feira',
    3: 'Quarta-feira',
    4: 'Quinta-feira',
    5: 'Sexta-feira',
    6: 'Sábado',
    7: 'Domingo'
}

clinics['TRIAL_START_DAY_NAME'] = clinics['TRIAL_START_DAY_OF_WEEK'].map(dias_da_semana)

In [ ]:
numerical_stats = clinics.describe()

categorical_distribution = clinics.describe(include=['object', 'bool'])

numerical_stats, categorical_distribution

#### Subscriptions

#### Limpeza e Preparação dos Dados

In [ ]:
date_columns = [
    'SUBSCRIPTION_START_DATE',
    'SUBSCRIPTION_END_DATE',
    'SUBSCRIPTION_CURRENT_PERIOD_STARTED_AT',
    'SUBSCRIPTION_CURRENT_PERIOD_ENDS_AT',
    'MOST_RECENT_INVOICE_CREATED_AT',
    'LAST_PAYMENT_AT'
]

for col in date_columns:
    subscriptions[col] = pd.to_datetime(subscriptions[col])

subscriptions.info()

In [ ]:
numerical_stats_subscription = subscriptions.describe()

categorical_distribution_subscription = subscriptions.describe(include=['object', 'bool'])

numerical_stats_subscription, categorical_distribution_subscription

In [ ]:
active_subscriptions = subscriptions[subscriptions['SUBSCRIPTION_STATUS'] == 'active'].shape[0]
cancelled_subscriptions = subscriptions[subscriptions['SUBSCRIPTION_STATUS'] == 'cancelled'].shape[0]

cancellation_rate = cancelled_subscriptions / subscriptions.shape[0]

subscriptions['SUBSCRIPTION_DURATION'] = (subscriptions['SUBSCRIPTION_END_DATE'] - subscriptions['SUBSCRIPTION_START_DATE']).dt.days
average_subscription_duration = subscriptions['SUBSCRIPTION_DURATION'].mean()

active_subscriptions, cancelled_subscriptions, cancellation_rate, average_subscription_duration

#### Activity

In [ ]:
activity['ACTIVITY_AT'] = pd.to_datetime(activity['ACTIVITY_AT'])

activity.info()

In [ ]:
numerical_stats_activity = activity.describe()

categorical_distribution_activity = activity.describe(include=['object', 'bool'])

numerical_stats_activity, categorical_distribution_activity

#### Análises - Modelo de Dados unindo as tabelas

Mesclando as tabelas

In [ ]:
join = pd.merge(clinics, subscriptions, on='CLINIC_ID', how='left', suffixes=('_clinic', '_subscription'))

join = pd.merge(join, activity, on='CLINIC_ID', how='left')

In [ ]:
join.info()

In [ ]:
base_final = join.groupby('CLINIC_ID').agg({
    'CLINIC_CREATED_AT': 'first',
    'INTEREST_CATEGORY_SIGNUP': 'first',
    'TRIAL_START_DATE': 'first',
    'TRIAL_END_DATE': 'first',
    'TRIAL_START_DAY_OF_WEEK': 'first',
    'TRIAL_START_DAY_OF_WEEK_CATEGORY':'first',
    'TRIAL_START_DAY_NAME':'first',
    'SUBSCRIPTION_START_DATE': 'first',
    'SUBSCRIPTION_END_DATE': 'first',
    'SUBSCRIPTION_STATUS': 'last',
    'ACTIVITY_AT': ['min', 'max'],
    'IS_VALID_ACTIVITY': 'sum',
    'IS_DELETION_ACTIVITY': 'sum'
}).reset_index()

base_final.columns = ['CLINIC_ID', 'CLINIC_CREATED_AT', 'INTEREST_CATEGORY_SIGNUP',  'TRIAL_START_DATE', 'TRIAL_END_DATE', 
                           'TRIAL_START_DAY_OF_WEEK','TRIAL_START_DAY_OF_WEEK_CATEGORY', 'TRIAL_START_DAY_NAME', 
                           'SUBSCRIPTION_START_DATE', 'SUBSCRIPTION_END_DATE', 'SUBSCRIPTION_STATUS', 'FIRST_ACTIVITY_DATE', 
                           'LAST_ACTIVITY_DATE', 'VALID_ACTIVITIES_COUNT', 'INVALID_ACTIVITIES_COUNT']

base_final.info()


Salvando arquivo na rede

In [ ]:
# Salvar a tabela agregada em CSV
base_final.to_csv('base_final_capim.csv', index=False, encoding='ansi')


Análise de Taxa de Conversão

In [ ]:
total_trial = len(base_final)
total_assinantes = base_final['SUBSCRIPTION_START_DATE'].notna().sum()
taxa_conversao = total_assinantes / total_trial * 100

print(f'Taxa de Conversão: {taxa_conversao:.2f}%')

Análise de Atividade

In [ ]:
atividades_validas_medias = base_final['VALID_ACTIVITIES_COUNT'].astype(float).mean()
atividades_invalidas_medias = base_final['INVALID_ACTIVITIES_COUNT'].astype(float).mean()

print(f'Atividades Válidas Médias: {atividades_validas_medias:.2f}')
print(f'Atividades Inválidas Médias: {atividades_invalidas_medias:.2f}')

Análise por Categoria de Interesse


In [ ]:
categorias_interesse = base_final['INTEREST_CATEGORY_SIGNUP'].unique()

for categoria in categorias_interesse:
    total_categoria = base_final[base_final['INTEREST_CATEGORY_SIGNUP'] == categoria]
    total_assinantes_categoria = total_categoria['SUBSCRIPTION_START_DATE'].notna().sum()
    taxa_conversao_categoria = total_assinantes_categoria / len(total_categoria) * 100
    
    print(f'{categoria}: {taxa_conversao_categoria:.2f}%')

#### Visualizações de Graficos

Visão Assinaturas


In [ ]:
import plotly.express as px

total_trial = len(base_final)
total_assinantes = base_final['SUBSCRIPTION_START_DATE'].notna().sum()
taxa_conversao = total_assinantes / total_trial * 100

fig = px.pie(values=[total_assinantes, total_trial - total_assinantes],
             names=['Assinantes', 'Não Assinantes'],
             title='Taxa de Conversão de Trials para Assinantes',
             color_discrete_sequence=px.colors.qualitative.Pastel)

fig.update_traces(textinfo='percent+label', 
                  pull=[0.1, 0], 
                  marker=dict(line=dict(color='black', width=2))) 

fig.update_layout(title_font_size=24, 
                  legend=dict(title='Status', title_font_size=14, font_size=12), 
                  annotations=[dict(text='Taxa de Conversão', x=0.5, y=0.5, font_size=20, showarrow=False)]) 

fig.show()

Atividade Durante o Trial

In [ ]:
atividades_validas_medias = base_final['VALID_ACTIVITIES_COUNT'].astype(float).mean()
atividades_invalidas_medias = base_final['INVALID_ACTIVITIES_COUNT'].astype(float).mean()

fig = px.bar(x=['Atividades Válidas Médias', 'Atividades Inválidas Médias'],
             y=[atividades_validas_medias, atividades_invalidas_medias],
             title='Atividades Médias por Trial',
             text_auto=True,
             labels={'x': 'Tipo de Atividade', 'y': 'Média de Atividades'})
fig.show()

Análise por Categoria de Interesse


In [ ]:
base_final['IS_SUBSCRIBED'] = base_final['SUBSCRIPTION_START_DATE'].notna()
conversion_rates = base_final.groupby('INTEREST_CATEGORY_SIGNUP')['IS_SUBSCRIBED'].mean() * 100

fig = px.bar(conversion_rates, x=conversion_rates.index, y=conversion_rates.values, title='Taxa de Conversão por Categoria de Interesse',
             text_auto=True,
             labels={'INTEREST_CATEGORY_SIGNUP': 'Categoria de Interesse', 'y': 'Taxa de Conversão (%)'})
fig.show()

Análise por Dia da Semana

In [ ]:
dias_da_semana_ordem = ['Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Sábado', 'Domingo']
base_final['TRIAL_START_DAY_NAME'] = pd.Categorical(base_final['TRIAL_START_DAY_NAME'], categories=dias_da_semana_ordem, ordered=True)

start_day_counts = base_final['TRIAL_START_DAY_NAME'].value_counts().sort_index()

fig = px.bar(start_day_counts, 
             x=start_day_counts.index, 
             y=start_day_counts.values, 
             title='Trials Iniciados por Dia da Semana',
             text_auto=True,
             labels={'TRIAL_START_DAY_NAME': 'Dia da Semana', 'y': 'Quantidade'})
fig.show()